# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [10]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-flash")

# Load the artifact from Lab 1
user_stories_str = load_artifact("day1_user_stories.json")
if user_stories_str:
    if isinstance(user_stories_str, str):
        user_stories_data = json.loads(user_stories_str)
    else:
        user_stories_data = user_stories_str
else:
    user_stories_data = []

2025-10-27 16:16:51,855 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-flash latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [11]:
# Create a prompt that instructs the LLM to act as a Product Manager
# and generate a PRD with three sections: Introduction, User Personas, and Features/User Stories
simple_prd_prompt = f"""
You are a Product Manager tasked with creating a Product Requirements Document.

Based on the following user stories, generate a comprehensive PRD with the following three sections:

1. **Introduction**: Provide a high-level overview of the product, its purpose, and the target users.

2. **User Personas**: Extract and describe the unique personas mentioned in the user stories, including their backgrounds and motivations.

3. **Features / User Stories**: List all the features derived from the user stories, organized logically.

User Stories Data:
{json.dumps(user_stories_data, indent=2)}

Format your response as clean markdown with clear headings and bullet points.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
## Product Requirements Document: New Hire Onboarding & Engagement Platform

---

### 1. Introduction

**Product Name:** New Hire Onboarding & Engagement Platform

**Overview:**
The New Hire Onboarding & Engagement Platform is a comprehensive digital solution designed to revolutionize the new employee onboarding experience. It aims to create a seamless, engaging, and efficient journey for new hires from pre-start to their initial weeks, while providing powerful tools for HR and L&D teams to manage, customize, and track the onboarding process. By centralizing information, digitizing paperwork, and fostering early connections, the platform ensures new employees feel prepared, valued, and integrated from day one.

**Purpose:**
The primary purpose of this platform is twofold:
1.  **For New Hires:** To provide a personalized, intuitive, and supportive environment that reduces pre-start anxiety, streamlines administrative tasks, and accelerates their integration

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [19]:
# Load the PRD template
prd_template_content = load_artifact("templates\prd_template.md")

# Create a prompt to populate the PRD template as a Senior Product Manager
template_prd_prompt = f"""
You are a Senior Product Manager tasked with creating a comprehensive Product Requirements Document.

You are provided with:
1. A PRD template that defines the structure and required sections
2. User stories data that contains the specific requirements

Your task is to populate the template with information derived from the user stories. For sections that aren't covered in the user stories (like Success Metrics, Out of Scope, Release Plan), use your professional judgment to create logical and appropriate content based on the overall project goals.

PRD Template:
{prd_template_content}

User Stories Data:
{json.dumps(user_stories_data, indent=2)}

Instructions:
1. Follow the EXACT structure of the template provided
2. Fill in every section of the template
3. Use the user stories to populate the "Functional Requirements & User Stories" section
4. For sections not explicitly covered in user stories, infer appropriate content based on the product's purpose
5. Ensure all placeholders and example text are replaced with actual content
6. Maintain professional documentation standards throughout

Generate the complete PRD as a markdown document.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\640109\AppData\Local\Temp\ipykernel_3148\2454316565.py:2: SyntaxWarning: invalid escape sequence '\p'
  prd_template_content = load_artifact("templates\prd_template.md")


--- Generating PRD from Template ---
# Product Requirements Document: New Hire Experience Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Product Team |
| **Version** | 1.0 |
| **Last Updated** | 2023-10-27 |

## 1. Executive Summary & Vision
The New Hire Experience Platform is designed to revolutionize our company's onboarding process. This product will centralize all pre-boarding and initial onboarding activities into a single, intuitive portal, addressing the current challenges of fragmented information and manual administrative tasks. Our vision is to create a seamless, engaging, and highly efficient onboarding journey for every new employee, enabling them to feel prepared, connected, and productive from day one, while significantly reducing the administrative burden on HR and hiring managers.

## 2. The Problem
A detailed look at the pain points this product will solve. This section justifies the project's existence.

**2.1. Problem Statement:**
New hires currently

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [23]:
# Create a prompt to generate a Pydantic model for the PRD
# This model will be used to programmatically validate PRD documents
pydantic_model_prompt = f"""
You are a Python developer tasked with creating a Pydantic model for validating Product Requirements Documents.

Based on the following PRD template structure, generate a Python Pydantic model that can represent and validate the structure of any PRD document.

PRD Template Structure:
{prd_template_content}

Requirements:
1. Create a Python class named 'ProductRequirementsDocument' that inherits from Pydantic's BaseModel
2. Add appropriate fields for each major section of the template (e.g., executive_summary, problem_statement, goals, user_stories, non_functional_requirements, etc.)
3. Use appropriate types from Python's typing library (str, List, Dict, Optional, etc.)
4. For sections with multiple items or complex data, use List or Dict types
5. For sections that might be optional, use Optional[]
6. Add helpful docstrings for each field explaining what it represents
7. Import all necessary modules at the top (from pydantic import BaseModel, etc.)
8. Make the model comprehensive to cover all sections in the template

Generate ONLY the Python code, without any markdown formatting or explanations. The code should be ready to save as a .py file.
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md")

--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Union
from datetime import date


class Persona(BaseModel):
    """Represents a user persona with their name and a typical scenario."""
    name: str = Field(..., description="The name of the user persona (e.g., The New Hire).")
    scenario: str = Field(..., description="A typical scenario or situation faced by this persona that highlights the problem.")


class GoalMetric(BaseModel):
    """Represents a product goal with its Key Performance Indicator (KPI) and target."""
    goal: str = Field(..., description="A specific, measurable goal for the product.")
    kpi: str = Field(..., description="The Key Performance Indicator used to measure the progress towards the goal.")
    target: str = Field(..., description="The measurable target value or outcome for the KPI.")


class UserStory(BaseModel):
    """Represents a functional requi

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.